In [18]:
import logging
import os

# LOGGING CONFIG ##############################
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
###############################################

In [19]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [20]:
'''
using the chest x-ray dataset from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/
check if the chest x-ray dataset exists
unzip the chest x-ray dataset if it exists
'''
if not os.path.exists('chest_xray'):
    if os.path.exists('chest-xray-pneumonia.zip'):
        logging.info('unzipping the dataset file')
        os.system('unzip chest-xray-pneumonia.zip')
        logging.info('unzipping is done')
        os.system('rm chest-xray-pneumonia.zip')
    else:
        logging.warning('please download the dataset from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/')

In [21]:
def create_data_from_image(dir, data_type, main, size):
    training_data = []
    occurrence = [0, 0, 0]
    for cat in main:
        logging.info('reading images for category {}'.format(cat))
        path =  os.path.join(dir, data_type, cat)
        label = main.index(cat)
        for img in os.listdir(path):
            # read and resize image
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_array = cv2.resize(img_array, (size, size))

                # append to training data
                occurrence[label] += 1
                training_data.append([img_array, label])

            except:
                logging.warn('error reading {}'.format(img))
    
    logging.info('reading images done')
    return training_data, occurrence

            
directory = 'chest_xray'
data_type = 'train'   # choose between train and test
main_category = ['NORMAL', 'PNEUMONIA']
image_size = 200

# create training data with labels: 0:normal, 1:bacterial 2:viral
training_data, occurrence = create_data_from_image(directory, data_type, main_category, image_size)

2020-05-14 15:30:03 INFO     reading images for category NORMAL
2020-05-14 15:30:33 INFO     reading images for category PNEUMONIA
2020-05-14 15:30:58 INFO     reading images done


In [22]:
# number of images with labels 0, 1
print('Normal:', occurrence[0], 'Pneumonia:', occurrence[1])
# weight is labels based on their occurence
weight = [float(i)/sum(occurrence) for i in occurrence]
print('Normal:', weight[0], 'Pneumonia:', weight[1])
class_weight = {0: weight[0],
                1: weight[1]}

# shuffle the training data otherwise the neural network model will be inefficient
import random 
random.shuffle(training_data)

Normal: 1341 Pneumonia: 3875
Normal: 0.2570935582822086 Pneumonia: 0.7429064417177914


In [23]:
# separate features and labels
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)

In [24]:
# we have to convert a list to a numpy array that is understandable for tensorflow
# -1 means everything in the list, 1 is because the image is gray scale
X = np.array(X).reshape(-1, image_size, image_size, 1)


In [25]:
# save the training data
# you need at least 8GB of ram for this
import pickle
pickle_out = open('trainings/X_2labels.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('trainings/y_2labels.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()

In [26]:
# import libraries to create neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

In [27]:
# uncomment the following to read X and y in case we want to re run from here
# this avoids the need to re-read all images
# import pickle
# X = pickle.load(open('trainings/X_2labels.pickle', 'rb'))
# y = pickle.load(open('trainings/y_2labels.pickle', 'rb'))

# in case of gray scale image data, we normalize it 
X = X/255.0 

In [28]:
import time
NAME = 'xray-2_labels-4_layers-2x32-{}'.format(int(time.time()))
# tensorboard = TensorBoard(log_dir='trainings/{}'.format(NAME))
# os.system("tensorboard --logdir='trainin/'")

In [34]:
# create the model 
model = Sequential() 

# layer 1
model.add(Conv2D(32, (4,4), strides=(4, 4), input_shape=X.shape[1:]))
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# layer 2
model.add(Conv2D(32, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))

# layer 3
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

# output layer, we use Dense(3) to have 3 labels 0,1,2
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 50, 50, 32)        544       
_________________________________________________________________
activation_16 (Activation)   (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 32)        16416     
_________________________________________________________________
activation_17 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                1179712   
_________________________________________________________________
activation_18 (Activation)   (None, 64)               

In [35]:
model.fit(X, y, batch_size=100, epochs=20, class_weight=class_weight, validation_split=0.1)
# model.fit(X, y, batch_size=100, epochs=20, class_weight=class_weight, validation_split=0.1, callbacks=[tensorboard])

Train on 4694 samples, validate on 522 samples
Epoch 1/20
4694/4694 [==============================] - 10s 2ms/sample - loss: 0.1737 - acc: 0.7857 - val_loss: 0.0876 - val_acc: 0.9502
Epoch 2/20
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.0688 - acc: 0.9188 - val_loss: 0.0381 - val_acc: 0.9693
Epoch 3/20
4694/4694 [==============================] - 10s 2ms/sample - loss: 0.0489 - acc: 0.9495 - val_loss: 0.0339 - val_acc: 0.9502
Epoch 4/20
4694/4694 [==============================] - 10s 2ms/sample - loss: 0.0440 - acc: 0.9550 - val_loss: 0.0301 - val_acc: 0.9636
Epoch 5/20
4694/4694 [==============================] - 10s 2ms/sample - loss: 0.0420 - acc: 0.9578 - val_loss: 0.0322 - val_acc: 0.9579
Epoch 6/20
4694/4694 [==============================] - 10s 2ms/sample - loss: 0.0325 - acc: 0.9646 - val_loss: 0.0248 - val_acc: 0.9770
Epoch 7/20
4694/4694 [==============================] - 9s 2ms/sample - loss: 0.0287 - acc: 0.9683 - val_loss: 0.0265 - val_acc: 0.9

In [36]:
# save the network weights
model.save_weights("trainings/{}.h5".format(NAME))